In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

Exception ignored in: <_io.FileIO name='data/20news/instances.pkl' mode='rb' closefd=True>


In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(11258, 1995)

# run

## initialize log

In [6]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [7]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 10000 0 0 

## run

In [8]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series    
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
   Time  Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0   477   0  0   1030   1031  1034  0.01  0.44  0.44  0.70  0.68
1   516   1  0    958    965   964  0.01  0.45  0.45  0.70  0.69
2   545   2  0    912    928   928  0.01  0.47  0.46  0.70  0.69
3   594   3  0    887    907   907  0.02  0.48  0.46  0.72  0.71
4   601   4  0    869    891   891  0.02  0.47  0.46  0.71  0.71
5   607   5  0    854    875   875  0.03  0.47  0.46  0.71  0.70
6   637   6  0    836    858   858  0.03  0.47  0.46  0.71  0.71
7   663   7  0    819    840   839  0.03  0.46  0.46  0.73  0.72
8   679   8  0    809    828   828  0.04  0.45  0.46  0.74  0.73
9   663   9  0    801    820   819  0.04  0.45  0.46  0.71  0.70
10  639  10  0    797    816   815  0.04  0.45  0.46  0.68  0.67
11  627  11  0    795    813   812  0.05  0.44  0.46  0.68  0.67
12  631  12  0    793    810   810  0.05  0.44  0.46  0.68  0.67
13  631  13  0    792    809   808  0.05  0.44  0.46  0.67  0.67
14  636  14  0    790    807   807  0.05  0.44  0.46  0.67  0.67
15  665  15  0    788    806   806  0.06  0.44  0.46  0.71  0.70
16  649  16  0    788    805   805  0.06  0.43  0.46  0.67  0.67
17  642  17  0    787    805   804  0.06  0.43  0.46  0.67  0.67
18  630  18  0    786    803   804  0.06  0.43  0.46  0.67  0.67
19  656  19  0    785    803   804  0.06  0.43  0.46  0.70  0.70
20  655  20  0    783    799   800  0.06  0.43  0.46  0.70  0.69
21  625  21  0    781    798   798  0.06  0.43  0.46  0.66  0.66
22  625  22  0    780    796   797  0.06  0.42  0.46  0.66  0.65
23  625  23  0    779    795   795  0.06  0.42  0.46  0.66  0.65
24  506  24  0    779    796   795  0.06  0.42  0.46  0.66  0.65
25  625  25  0    778    795   794  0.06  0.42  0.46  0.66  0.65
26  651  26  0    777    794   793  0.06  0.43  0.46  0.70  0.68
27  613  27  0    776    793   792  0.06  0.42  0.46  0.66  0.65
28  615  28  0    775    792   791  0.06  0.42  0.46  0.66  0.64
29  623  29  0    775    791   791  0.06  0.42  0.46  0.66  0.64
30  630  30  0    775    791   790  0.06  0.42  0.46  0.66  0.64
31  644  31  0    774    790   789  0.06  0.42  0.46  0.66  0.64
32  626  32  0    773    789   788  0.06  0.42  0.46  0.66  0.64
33  634  33  0    773    788   787  0.06  0.42  0.46  0.66  0.64
34  643  34  0    772    787   787  0.06  0.42  0.46  0.66  0.64
35  627  35  0    772    787   787  0.06  0.42  0.46  0.65  0.64
36  489  36  0    772    788   787  0.06  0.42  0.46  0.65  0.64
37  486  37  0    772    787   787  0.06  0.42  0.46  0.65  0.64
38  599  38  0    771    786   785  0.06  0.42  0.46  0.65  0.64
39  606  39  0    770    786   785  0.06  0.42  0.46  0.65  0.64
40  610  40  0    769    785   784  0.06  0.42  0.46  0.65  0.64
41  606  41  0    769    785   784  0.06  0.42  0.46  0.65  0.64
42  611  42  0    769    784   783  0.06  0.42  0.46  0.65  0.64
43  612  43  0    768    783   783  0.06  0.42  0.46  0.65  0.64
44  606  44  0    767    783   782  0.06  0.42  0.46  0.65  0.64
45  611  45  0    766    782   781  0.06  0.42  0.47  0.65  0.64
46  611  46  0    766    781   781  0.06  0.42  0.46  0.65  0.64
47  610  47  0    765    781   781  0.06  0.42  0.46  0.65  0.64
48  580  48  0    765    781   781  0.06  0.42  0.46  0.65  0.64
49  582  49  0    764    780   780  0.06  0.42  0.46  0.65  0.64

Exception ignored in: <_io.FileIO name='model/20news/ncrp/20news-mncrp-alp10.50.1-eta10-gam0.0001-epoch50/log' mode='wb' closefd=True>


 0 11258 384451.0 write one article say know get think like people make
   0-1 3848 176028.0 people say go government one state make gun get write
     0-1-1 2310 13650.0 car drive bike ride dog road insurance dealer buy speed
     0-1-2 438 860.0 ac ted slave mon document percent ad frank editor ed
     0-1-3 264 1128.0 insurance health medical hospital private canadian care card canada doctor
     0-1-4 142 201.0 db hang mr period tear matthew arrive western origin plot
     0-1-5 611 652.0 nsa minority andrew secret color privacy value la signature society
     0-1-6 83 224.0 oo scott bruce student apr bear cult mail identify claim
   0-2 4238 196786.0 use file get drive program window work write system run
     0-2-1 3985 6292.0 car engine bike battery condition model sell ac sale channel
     0-2-2 179 179.0 doug apr western copyright douglas escape secure library structure error
     0-2-3 74 57.0 nt toolkit annual discussion vote issue brian entirely status conference
   0-3 155